In [ ]:
! pip3 install bayesian_optimization
! pip3 install xgboost

In [1]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
train_values = pd.read_csv('Archivos auxiliares KNN/train_values_short1.csv', index_col='building_id')
train_labels = pd.read_csv('Archivos auxiliares KNN/train_labels.csv', index_col='building_id')

In [177]:
def target(resultado):
    return resultado['target']

def mostrar(resultados, min=0):
    
    a_borrar = []
    for resultado in resultados:
        if(resultado['target']<min):
            a_borrar.append(resultado)
    resultados_importantes = [a for a in resultados if a not in a_borrar]
    
    resultados_importantes.sort(reverse=True, key=target)
    
    for resultado in resultados_importantes:
        print("target: " , resultado['target'])
        for key,value in resultado['params'].items():
            print("\t\t", key, ':', value)
        print()

In [178]:
def funcion_a_optimizar1(max_depth, min_child_weight):
    x = train_values
    le = LabelEncoder()
    y = le.fit_transform(y=train_labels.values.ravel())
    xgb = XGBClassifier( objective = "multi:softmax", use_label_encoder=False, seed=10,
                         max_depth=round(max_depth),
                         min_child_weight=min_child_weight,
                         gamma=0,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         learning_rate =0.1, 
                         n_estimators=240, 
                         )
                       
    return cross_val_score(xgb, x, y, scoring='f1_micro', cv=10).mean()

In [179]:
limites = {"max_depth":(3,11), "min_child_weight":(1,8) }

In [182]:
optimizer = BayesianOptimization(
    f=funcion_a_optimizar1,
    random_state=9,
    verbose=5,
    pbounds=limites,
)

load_logs(optimizer, "./anteriores.json")
logger = JSONLogger(path="./logs_nuevos.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [183]:
optimizer.maximize(
    init_points=5,
    n_iter=40,
)

[20:25:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:26:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [184]:
mostrar(optimizer.res)

target:  0.7585197330134774
		 max_depth : 8.399244502867223
		 min_child_weight : 1.1037597467970075



In [3]:
 xgb = XGBClassifier( objective = "multi:softmax", use_label_encoder=False, seed=10,
                         gamma=0,
                         subsample=0.8,
                         colsample_bytree=0.8,
                         learning_rate =0.1, 
                         n_estimators=240, 
                        max_depth = 9,
                        min_child_weight = 6.0,
                        num_class=3, 
                        verbose=0
                         )

In [3]:
    le = LabelEncoder()
    y = le.fit_transform(y=train_labels.values.ravel())

In [6]:
xgb.fit(train_values, y)

[10:39:24] WARNING: ../src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:39:25] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=6.0, missing=nan, monotone_constraints='()',
              n_estimators=240, n_jobs=4, num_class=3, num_parallel_tree=1,
              objective='multi:softprob', random_state=10, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=10, subsample=0.8,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbose=0, ...)

In [8]:
test = pd.read_csv("Archivos auxiliares KNN/test_values_short1.csv", index_col='building_id')

In [9]:
preds = xgb.predict(test)
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('submissionxg.csv')

In [8]:
# Soy un gil, me olvide de encodear devuelta jejejeje
submission_horrible = pd.read_csv('submissionxg.csv', index_col='building_id')

In [9]:
submission_mejorada = submission_horrible + 1

In [12]:
submission_mejorada.to_csv('subs/xgboost_short1_001.csv', index=True)